https://towardsdatascience.com/character-level-cnn-with-keras-50391c3adf33
https://www.kaggle.com/kmader/character-level-cnn-classification-with-dilations

In [ ]:
import tensorflow as tf
import json
# import keras
# from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import tensorflow as tf
from tensorflow.python.keras.layers import *
from tensorflow.python import keras

import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_text as text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
blog_posts_data_dir = "/content/drive/MyDrive/datasets/BT-AP-19 Corpus/all/"
# blog_posts_data_dir = "/content/drive/MyDrive/datasets/"
train_file_name = "train.json"
test_file_name = "test.json"

# Load data
with open(blog_posts_data_dir + train_file_name) as r:
    training_set = json.load(r)
training_set = training_set[:400000]
training_set[0]

{'age': '50', 'gender': 'female', 'post': 'Umar Murtaza: Head of SM Operation'}

In [ ]:
def get_gender_as_num(gender):
    if gender == "male":
        return 0
    else:
        return 1

In [ ]:
def get_age_group(age): # HIGH NOTE: changing each of the scalars to a vector. This is probably not a good idea
    if age < 18:
        # 13 - 17
        return [1, 0, 0]
    elif age < 28:
        # 23 - 27
        return [0, 1, 0]
    elif age < 49:
        # 33 - 48
        return [0, 0, 1]
    else:
        return [0, 0, 0]

In [ ]:
def get_age_group(age): # HIGH NOTE: changing each of the scalars to a vector. This is probably not a good idea
    age = int(age)
    if age == 18:
        # 13 - 17
        return [1, 0, 0, 0]
    elif age == 25:
        # 23 - 27
        return [0, 1, 0, 0]
    elif age == 35:
        # 33 - 48
        return [0, 0, 1, 0]
    elif age == 50:
        # 33 - 48
        return [0, 0, 0, 1]
    else:
        return [0, 0, 0, 0]

In [ ]:
posts = [instance["post"] for instance in training_set]
len(posts)

33637

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [ ]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(posts)

In [ ]:
len(tk.word_index)
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
len(tk.word_index)

69

In [ ]:
sequences = tk.texts_to_sequences(posts)
median_words_per_tokenized_sample = np.median([len(post) for post in sequences])
median_words_per_tokenized_sample

55.0

In [ ]:
data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen = int(median_words_per_tokenized_sample),
                                                     padding = "post")
for i, instance in enumerate(training_set):
    instance["post"] = data[i]
    instance["gender"] = get_gender_as_num(instance["gender"])
    instance["age"] = get_age_group(int(instance["age"]))

In [ ]:
training_set[0]

{'age': [0, 0, 0, 1],
 'gender': 1,
 'post': array([21, 13,  1, 18, 69, 13, 21, 18, 20,  1, 26,  1, 42, 69,  8,  5,  1,
         4, 69, 15,  6, 69, 19, 13, 69, 15, 16,  5, 18,  1, 20,  9, 15, 14,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0], dtype=int32)}

In [ ]:
input_size = int(median_words_per_tokenized_sample)
vocab_size = len(tk.word_index)

In [ ]:
embedding_size = 100#len(tk.word_index)

In [ ]:
conv_layers = [[256, 7, 3],
            #    [256, 7, 3],
            #    [256, 3, -1],
            #    [256, 3, -1],
            #    [256, 3, -1],
               [256, 3, 3]]

In [ ]:
fully_connected_layers = [1024, 1024]
num_of_classes = 4
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [ ]:
# Embedding weights
embedding_weights = []  # (70, 69)
embedding_weights.append(np.zeros(vocab_size))  # (0, 69)

for char, i in tk.word_index.items():  # from index 1 to 69
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
embedding_weights.shape

(70, 69)

In [ ]:
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size)

inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
x = embedding_layer(inputs)
# Conv
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  # Adam, categorical_crossentropy
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 55)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 55, 100)           7000      
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 49, 256)           179456    
_________________________________________________________________
activation_11 (Activation)   (None, 49, 256)           0         
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 16, 256)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 14, 256)           196864    
_________________________________________________________________
activation_12 (Activation)   (None, 14, 256)           0   

In [ ]:
posts_train = np.array([instance["post"] for instance in training_set])
ages_train = np.array([instance["age"] for instance in training_set])

In [ ]:
history_1 = model.fit(posts_train, ages_train, epochs = 10, batch_size = 500, validation_split = 0.2)

Epoch 1/10
54/54 [==============================] - 33s 47ms/step - loss: 1.3131 - accuracy: 0.3229 - val_loss: 1.2509 - val_accuracy: 0.3934
Epoch 2/10
54/54 [==============================] - 2s 35ms/step - loss: 1.2433 - accuracy: 0.3970 - val_loss: 1.2207 - val_accuracy: 0.4203
Epoch 3/10
54/54 [==============================] - 2s 35ms/step - loss: 1.1865 - accuracy: 0.4350 - val_loss: 1.1887 - val_accuracy: 0.4410
Epoch 4/10
54/54 [==============================] - 2s 35ms/step - loss: 1.1114 - accuracy: 0.4890 - val_loss: 1.1948 - val_accuracy: 0.4328
Epoch 5/10
54/54 [==============================] - 2s 36ms/step - loss: 1.0232 - accuracy: 0.5456 - val_loss: 1.3044 - val_accuracy: 0.4348
Epoch 6/10
54/54 [==============================] - 2s 36ms/step - loss: 0.9031 - accuracy: 0.6040 - val_loss: 1.2768 - val_accuracy: 0.4431
Epoch 7/10
54/54 [==============================] - 2s 35ms/step - loss: 0.7426 - accuracy: 0.6831 - val_loss: 1.4019 - val_accuracy: 0.4392
Epoch 8/10
5

In [ ]:
def build_model(conv_layers = 2, 
                dilation_rates = [0, 2, 4, 8, 16], 
                embed_size = 256):
    inp = Input(shape=(None, ))
    x = Embedding(input_dim = len(tk.word_counts)+1, 
                  output_dim = embed_size)(inp)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []
    # dilation rate lets us use ngrams and skip grams to process 
    for dilation_rate in dilation_rates:
        x = prefilt_x
        for i in range(2):
            if dilation_rate>0:
                x = Conv1D(16*2**(i), 
                           kernel_size = 3, 
                           dilation_rate = dilation_rate,
                          activation = 'relu',
                          name = 'ngram_{}_cnn_{}'.format(dilation_rate, i)
                          )(x)
            else:
                x = Conv1D(16*2**(i), 
                           kernel_size = 1,
                          activation = 'relu',
                          name = 'word_fcl_{}'.format(i))(x)
        out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    x = concatenate(out_conv, axis = -1)    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(num_of_classes, activation='softmax')(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss=loss,
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

model2 = build_model(dilation_rates = [0, 2, 4, 8])
model2.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, None, 256)    144384      input_7[0][0]                    
__________________________________________________________________________________________________
dropout_47 (Dropout)            (None, None, 256)    0           embedding_11[0][0]               
__________________________________________________________________________________________________
word_fcl_0 (Conv1D)             (None, None, 16)     4112        dropout_47[0][0]                 
____________________________________________________________________________________________

In [ ]:
history_1 = model2.fit(posts_train, ages_train, epochs = 20, batch_size = 500, validation_split = 0.2)

Epoch 1/20
54/54 [==============================] - 5s 63ms/step - loss: 1.3448 - accuracy: 0.2828 - val_loss: 1.2906 - val_accuracy: 0.3401
Epoch 2/20
54/54 [==============================] - 3s 59ms/step - loss: 1.3038 - accuracy: 0.3263 - val_loss: 1.2802 - val_accuracy: 0.3830
Epoch 3/20
54/54 [==============================] - 3s 58ms/step - loss: 1.2908 - accuracy: 0.3587 - val_loss: 1.2556 - val_accuracy: 0.4058
Epoch 4/20
54/54 [==============================] - 3s 58ms/step - loss: 1.2678 - accuracy: 0.3787 - val_loss: 1.2422 - val_accuracy: 0.4116
Epoch 5/20
54/54 [==============================] - 3s 57ms/step - loss: 1.2557 - accuracy: 0.3903 - val_loss: 1.2321 - val_accuracy: 0.4095
Epoch 6/20
54/54 [==============================] - 3s 57ms/step - loss: 1.2473 - accuracy: 0.3953 - val_loss: 1.2255 - val_accuracy: 0.4206
Epoch 7/20
54/54 [==============================] - 3s 57ms/step - loss: 1.2367 - accuracy: 0.4059 - val_loss: 1.2234 - val_accuracy: 0.4185
Epoch 8/20
54

In [ ]:
# Load data
with open(blog_posts_data_dir + test_file_name) as r:
    test_set = json.load(r)

test_posts = [instance["post"] for instance in test_set]
test_sequences = tk.texts_to_sequences(test_posts)
test_post_data = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen = int(median_words_per_tokenized_sample),
                                                     padding = "post")
for i, instance in enumerate(test_set):
    instance["post"] = test_post_data[i]
    instance["gender"] = get_gender_as_num(instance["gender"])
    instance["age"] = get_age_group(int(instance["age"]))

posts_test = np.array([instance["post"] for instance in test_set])
ages_test = np.array([instance["age"] for instance in test_set])

print(model.evaluate(posts_test, ages_test))

from sklearn.metrics import classification_report
pred = model2.predict(posts_test)
print(classification_report(np.argmax(ages_test, axis=1), np.argmax(pred, axis=1)))

263/263 [==============================] - 1s 3ms/step - loss: 1.9699 - accuracy: 0.4421
[1.9698514938354492, 0.44209274649620056]
              precision    recall  f1-score   support

           0       0.47      0.60      0.53      2698
           1       0.44      0.28      0.34      2647
           2       0.38      0.52      0.44      2309
           3       0.37      0.05      0.09       756

    accuracy                           0.43      8410
   macro avg       0.42      0.36      0.35      8410
weighted avg       0.43      0.43      0.40      8410



##Gender

In [ ]:
def build_model(conv_layers = 2, 
                dilation_rates = [0, 2, 4, 8, 16], 
                embed_size = 256):
    inp = Input(shape=(None, ))
    x = Embedding(input_dim = len(tk.word_counts)+1, 
                  output_dim = embed_size)(inp)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []
    # dilation rate lets us use ngrams and skip grams to process 
    for dilation_rate in dilation_rates:
        x = prefilt_x
        for i in range(2):
            if dilation_rate>0:
                x = Conv1D(16*2**(i), 
                           kernel_size = 3, 
                           dilation_rate = dilation_rate,
                          activation = 'relu',
                          name = 'ngram_{}_cnn_{}'.format(dilation_rate, i)
                          )(x)
            else:
                x = Conv1D(16*2**(i), 
                           kernel_size = 1,
                          activation = 'relu',
                          name = 'word_fcl_{}'.format(i))(x)
        out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    x = concatenate(out_conv, axis = -1)    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

model3 = build_model(dilation_rates = [0, 2, 4, 8])
model3.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, None, 256)    144384      input_9[0][0]                    
__________________________________________________________________________________________________
dropout_62 (Dropout)            (None, None, 256)    0           embedding_13[0][0]               
__________________________________________________________________________________________________
word_fcl_0 (Conv1D)             (None, None, 16)     4112        dropout_62[0][0]                 
___________________________________________________________________________________________

In [ ]:
posts_train = np.array([instance["post"] for instance in training_set])
genders_train = np.array([instance["gender"] for instance in training_set])

In [ ]:
history_1 = model3.fit(posts_train, genders_train, epochs = 10, batch_size = 500, validation_split = 0.2)

Epoch 1/10
54/54 [==============================] - 5s 62ms/step - loss: 0.6428 - accuracy: 0.6625 - val_loss: 0.6156 - val_accuracy: 0.6806
Epoch 2/10
54/54 [==============================] - 3s 57ms/step - loss: 0.6092 - accuracy: 0.6875 - val_loss: 0.6030 - val_accuracy: 0.6806
Epoch 3/10
54/54 [==============================] - 3s 57ms/step - loss: 0.6033 - accuracy: 0.6826 - val_loss: 0.5968 - val_accuracy: 0.6806
Epoch 4/10
54/54 [==============================] - 3s 57ms/step - loss: 0.5952 - accuracy: 0.6808 - val_loss: 0.5913 - val_accuracy: 0.6806
Epoch 5/10
54/54 [==============================] - 3s 57ms/step - loss: 0.5882 - accuracy: 0.6858 - val_loss: 0.5913 - val_accuracy: 0.6821
Epoch 6/10
54/54 [==============================] - 3s 57ms/step - loss: 0.5888 - accuracy: 0.6830 - val_loss: 0.5834 - val_accuracy: 0.6949
Epoch 7/10
54/54 [==============================] - 3s 57ms/step - loss: 0.5818 - accuracy: 0.6903 - val_loss: 0.5801 - val_accuracy: 0.6901
Epoch 8/10
54

In [ ]:
posts_test = np.array([instance["post"] for instance in test_set])
genders_test = np.array([instance["gender"] for instance in test_set])
print(model3.evaluate(posts_test, genders_test))

from sklearn.metrics import classification_report
pred = model3.predict(posts_test)
print(classification_report(genders_test, (pred > 0.5).astype(int)))

263/263 [==============================] - 1s 3ms/step - loss: 0.5591 - accuracy: 0.7190
[0.5591228604316711, 0.7190249562263489]
              precision    recall  f1-score   support

           0       0.66      0.16      0.26      2565
           1       0.72      0.96      0.83      5845

    accuracy                           0.72      8410
   macro avg       0.69      0.56      0.54      8410
weighted avg       0.70      0.72      0.65      8410

